In [1]:
import os
os.getenv("SLURM_TMPDIR")

'/scratch/local/64463386'

In [2]:
os.chdir(os.getenv('SLURM_TMPDIR'))

In [3]:
#os.mkdir('/blue/adamginsburg/adamginsburg/almaimf/test_reconstruction')
os.chdir('/blue/adamginsburg/adamginsburg/almaimf/test_reconstruction')

# Restore non-JvM cubes from distributed data

We used JvM (https://ui.adsabs.harvard.edu/abs/1995AJ....110.2037J/abstract) correction to produce the distributed data cubes.  

In the JvM correction process, the residuals are suppressed by a factor set by the ratio of the dirty beam within the first null to the clean beam.

If you want to restore the residuals to their original values, this script demonstrates how, starting with the data download:

## Download data:
Download some data using the script at https://gist.github.com/keflavich/46cbf66a4c5ba7a9d64753acdfeeb036

In [4]:
# An API key on dataverse is required
with open(os.path.expanduser('~/.almaimf_dataverse_token'), 'r') as fh:
    api_key = token = fh.read().strip()

You'll want to replace G353.41_B6_spw1 with whatever field name you are looking at.  This example case is to demonstrate the process and prove it works

In [5]:
field_id = 'G353.41_B6_spw1_12M_spw1'

In [6]:
import requests, os

dataverse_server='https://dataverse.harvard.edu'

suffix = {'model': 'YWW5BY',
          'pb': 'RBS6KT',
          'cont': 'SF09HK',
          'cube': 'CJ3YXU'}

for doiend in suffix.values():
    persistentId = f'doi:10.7910/DVN/{doiend}'

    url_list = f'{dataverse_server}/api/datasets/:persistentId/versions/:draft/files?key={api_key}&persistentId={persistentId}'
    filelist_resp = requests.get(url_list)
    file_metadata = filelist_resp.json()

    # loop through *all files* and download them if they're not already downloaded
    for fm in file_metadata['data']:
        if field_id in fm['dataFile']['filename'] and not os.path.exists(fm['dataFile']['filename']):
            print("Downloading ", fm['dataFile']['filename'])
            resp = requests.get(f"{dataverse_server}/api/access/datafile/{fm['dataFile']['id']}", stream=True,
                                params={'key': api_key},
                                auth=(api_key, ''), )
            resp.raise_for_status()
            with open(fm['dataFile']['filename'], 'wb') as fh:
                fh.write(resp.content)

# Reconstruct more cubes

We'll be making several copies of the cubes with different properties:

 * non-continuum-subtracted
 * residual
 * non-PB-corrected
 * non-continuum-subtracted and non-pb-corrected
 * non-JvM-corrected
 
Each of these cubes will take up significant hard drive space, and some of these operations are computationally expensive and require a lot of memory.

In [7]:
from spectral_cube import SpectralCube

In [8]:
cube = SpectralCube.read(f'{field_id}.JvM.image.pbcor.statcont.contsub.fits', use_dask=True)
cube.allow_huge_operations = True
cube

DaskSpectralCube with shape=(959, 823, 819) and unit=Jy / beam and chunk size (322, 322, 273):
 n_x:    819  type_x: RA---SIN  unit_x: deg    range:   262.586003 deg:  262.632997 deg
 n_y:    823  type_y: DEC--SIN  unit_y: deg    range:   -34.716545 deg:  -34.677728 deg
 n_s:    959  type_s: FREQ      unit_s: Hz     range: 217045473750.146 Hz:217279338345.200 Hz

In [9]:
for com in cube.header['COMMENT']:
    if 'JvM_epsilon_median' in com:
        JvM_Factor = float(com.split("=")[-1])
JvM_Factor

0.7887069317340462

In [10]:
from astropy.io import fits
from astropy import units as u

In [11]:
pb = fits.open(f'{field_id}.flatpb.fits')

In [12]:
non_pb_cube = cube * pb[0].data
non_pb_cube.write(f'{field_id}.JvM.image.statcont.contsub.fits', overwrite=True)

In [13]:
cont = fits.open(f'{field_id}.JvM.image.pbcor.statcont.cont.fits')

In [14]:
non_contsub_cube = cube + cont[0].data * u.Unit(cont[0].header['BUNIT'])
non_contsub_cube.write(f'{field_id}.JvM.image.pbcor.fits', overwrite=True)
non_contsub_non_pb_cube = non_contsub_cube * pb[0].data
non_contsub_non_pb_cube.write(f'{field_id}.JvM.image.fits', overwrite=True)

In [15]:
modcube = SpectralCube.read(f'{field_id}.model.minimized.fits.gz', use_dask=True)
modcube.allow_huge_operations = True
modcube

DaskSpectralCube with shape=(959, 823, 819) and unit=Jy / pix and chunk size (322, 322, 273):
 n_x:    819  type_x: RA---SIN  unit_x: deg    range:   262.586003 deg:  262.632997 deg
 n_y:    823  type_y: DEC--SIN  unit_y: deg    range:   -34.716545 deg:  -34.677728 deg
 n_s:    959  type_s: FREQ      unit_s: Hz     range: 217045473750.146 Hz:217279338345.200 Hz

In [16]:
import radio_beam

## Creating the Residual

This step makes the original residual cube.  It is expensive because it requires convolving every channel of the model cube with the target cube beam.

You may need to explore some of spectral-cube's options for memory-intensive operations and/or use a more powerful computer for this step.

Note also that we're using a bit of a hack to make the model cube, with units of Jy/pixel, convolvable to the target beam.

In [17]:
pixsize = modcube.wcs.proj_plane_pixel_area()**0.5
# beam size needs to be scaled such that the conversion factor is 1
modcube_with_deltabeam = modcube.with_beam(radio_beam.Beam(pixsize / 1.1330900354567983))
modcube_with_jybeam = modcube_with_deltabeam.to(u.Jy/u.beam)

In [18]:
modcube_smooth = modcube_with_jybeam.convolve_to(cube.beam)

In [ ]:
residual = (non_contsub_non_pb_cube - modcube_smooth) / JvM_Factor
residual.write(f'{field_id}.residual.fits', overwrite=True)

In [ ]:
non_jvm_cube = non_contsub_non_pb_cube + (residual * (1-JvM_Factor))
non_jvm_cube.write(f'{field_id}.image.fits', overwrite=True)
non_jvm_pbcor_cube = non_jvm_cube / pb[0].data
non_jvm_pbcor_cube.write(f'{field_id}.image.pbcor.fits', overwrite=True)

## Validation

For the example case, the peak value is at X,Y,Z= and has value=

In [ ]:
non_jvm_cube.argmax()